In [66]:
import keras
import pandas as pd
import numpy as np
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.models import *

In [67]:
data = pd.read_csv('../data/files/multiple_concatenated_tickers.csv')
data.head()

,Index,AAPL.Open,AAPL.High,AAPL.Low,AAPL.Close,AAPL.Volume,AAPL.Adjusted,TSLA.Open,TSLA.High,TSLA.Low,...,AMZN.Low,AMZN.Close,AMZN.Volume,AMZN.Adjusted,NFLX.Open,NFLX.High,NFLX.Low,NFLX.Close,NFLX.Volume,NFLX.Adjusted
0,2007-01-03,12.327143,12.368571,11.700000,11.971429,309579900.0,7.982585,0.0,0.0,0.0,...,38.049999,38.700001,12405100,38.700001,3.714286,3.824286,3.677143,3.801429,16440900.0,3.801429
1,2007-01-04,12.007143,12.278571,11.974286,12.237143,211815100.0,8.159763,0.0,0.0,0.0,...,38.259998,38.900002,6318400,38.900002,3.772857,3.828571,3.585714,3.621428,15959300.0,3.621428
2,2007-01-05,12.252857,12.314285,12.057143,12.150000,208685400.0,8.101658,0.0,0.0,0.0,...,37.599998,38.369999,6619700,38.369999,3.620000,3.620000,3.492857,3.544286,15190700.0,3.544286
3,2007-01-08,12.280000,12.361428,12.182858,12.210000,199276700.0,8.141665,0.0,0.0,0.0,...,37.169998,37.500000,6783000,37.500000,3.545714,3.555714,3.367143,3.404286,18344900.0,3.404286
4,2007-01-09,12.350000,13.282857,12.164286,13.224286,837324600.0,8.817995,0.0,0.0,0.0,...,37.340000,37.779999,5703000,37.779999,3.427143,3.440000,3.360000,3.427143,10611300.0,3.427143


In [68]:
data_mat = data.iloc[:,1:].as_matrix()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [69]:
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

In [70]:
seq_len = 5
data = np.array(normalise_windows(data_mat))

In [71]:
sequence_length = seq_len + 1
result = []
for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])

# if normalise_window:
#     result = normalise_windows(result)

result = np.array(result)

row = round(0.9 * result.shape[0])
train = result[:int(row), :]
np.random.shuffle(train)
x_train = train[:, :-1]
y_train = train[:, -1]
y_train = y_train[:,1] # Extract AAPL Only
x_test = result[int(row):, :-1]
y_test = result[int(row):, -1]
y_test = y_test[:,1] # Extract AAPL Only

# x_train = np.reshape(x_train, (1, x_train.shape[0], x_train.shape[1]))
# x_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1]))  

[x_train, y_train, x_test, y_test]

[array([[[ 0.00000000e+00,  1.17020166e-02, -7.41827403e-03, ...,
          -7.53630715e-01,  6.74203374e+05, -7.53630715e-01],
         [ 0.00000000e+00,  5.48062805e-03, -1.82335683e-02, ...,
          -7.53583479e-01,  4.04424582e+05, -7.53583479e-01],
         [ 0.00000000e+00,  0.00000000e+00, -2.08090035e-02, ...,
          -7.51035136e-01,  4.96553847e+05, -7.51035136e-01],
         [ 0.00000000e+00,  4.24261755e-03, -1.62282980e-02, ...,
          -7.54030568e-01,  3.95044592e+05, -7.54030568e-01],
         [ 0.00000000e+00,  1.16852564e-03, -1.18971748e-02, ...,
          -7.50584207e-01,  5.38613846e+05, -7.50584207e-01]],
 
        [[ 0.00000000e+00,  1.09212885e-03, -8.42500182e-03, ...,
          -2.63448245e-01,  1.14538715e+04, -2.63448245e-01],
         [ 0.00000000e+00,  9.04044424e-03, -3.09277562e-03, ...,
          -2.59068767e-01,  2.15850432e+04, -2.59068767e-01],
         [ 0.00000000e+00,  6.45516025e-03, -5.19559163e-03, ...,
          -2.49997176e-01,  1.80411

In [74]:
LAYERS = 25
model = Sequential()

model.add(LSTM(
    input_dim=36,
    output_dim=LAYERS,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    LAYERS,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='adam')
print('compilation time : ', time.time() - start)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  import sys
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 36), units=25)`
  import sys


compilation time :  0.021585941314697266


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  app.launch_new_instance()


In [118]:
MODELNAME = 'multiplemodeltest'
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./forecast/models/'+MODELNAME+'_best.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, verbose=1, epsilon=1e-4, mode='min')

/usr/local/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [120]:
VALIDATIONSIZE = 0.3
EPOCHS = 1000
history = model.fit(
    x_train,
    y_train,
    batch_size=512,
    nb_epoch=EPOCHS,
    validation_split=VALIDATIONSIZE,
    callbacks = [reduce_lr_loss, earlyStopping])

Train on 1946 samples, validate on 835 samples
Epoch 1/1000
1946/1946 [==============================] - 0s 98us/step - loss: 0.0023 - val_loss: 2.3942e-04


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 2/1000
1946/1946 [==============================] - 0s 68us/step - loss: 0.0021 - val_loss: 2.4030e-04
Epoch 3/1000
1946/1946 [==============================] - 0s 50us/step - loss: 0.0022 - val_loss: 2.4179e-04
Epoch 4/1000
1946/1946 [==============================] - 0s 52us/step - loss: 0.0021 - val_loss: 2.4060e-04
Epoch 5/1000
1946/1946 [==============================] - 0s 58us/step - loss: 0.0021 - val_loss: 2.3758e-04
Epoch 6/1000
1946/1946 [==============================] - 0s 57us/step - loss: 0.0022 - val_loss: 2.3510e-04
Epoch 7/1000
1946/1946 [==============================] - 0s 55us/step - loss: 0.0021 - val_loss: 2.3287e-04
Epoch 8/1000
1946/1946 [==============================] - 0s 51us/step - loss: 0.0021 - val_loss: 2.3215e-04
Epoch 9/1000
1946/1946 [==============================] - 0s 55us/step - loss: 0.0020 - val_loss: 2.3064e-04
Epoch 10/1000
1946/1946 [==============================] - 0s 69us/step - loss: 0.0021 - val_loss: 2.2969e-04
Epoch 11/1000
1946

1946/1946 [==============================] - 0s 60us/step - loss: 0.0016 - val_loss: 2.0990e-04
Epoch 75/1000
1946/1946 [==============================] - 0s 63us/step - loss: 0.0015 - val_loss: 2.0979e-04
Epoch 76/1000
1946/1946 [==============================] - 0s 58us/step - loss: 0.0016 - val_loss: 2.0970e-04
Epoch 77/1000
1946/1946 [==============================] - 0s 74us/step - loss: 0.0017 - val_loss: 2.0962e-04
Epoch 78/1000
1946/1946 [==============================] - 0s 61us/step - loss: 0.0017 - val_loss: 2.0955e-04
Epoch 79/1000
1946/1946 [==============================] - 0s 60us/step - loss: 0.0016 - val_loss: 2.0949e-04
Epoch 80/1000
1946/1946 [==============================] - 0s 50us/step - loss: 0.0016 - val_loss: 2.0940e-04
Epoch 81/1000
1946/1946 [==============================] - 0s 47us/step - loss: 0.0014 - val_loss: 2.0932e-04

Epoch 00081: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 82/1000
1946/1946 [=============================

1946/1946 [==============================] - 0s 59us/step - loss: 0.0015 - val_loss: 2.0822e-04
Epoch 146/1000
1946/1946 [==============================] - 0s 62us/step - loss: 0.0014 - val_loss: 2.0821e-04
Epoch 147/1000
1946/1946 [==============================] - 0s 72us/step - loss: 0.0015 - val_loss: 2.0820e-04
Epoch 148/1000
1946/1946 [==============================] - 0s 58us/step - loss: 0.0015 - val_loss: 2.0820e-04
Epoch 149/1000
1946/1946 [==============================] - 0s 51us/step - loss: 0.0015 - val_loss: 2.0820e-04
Epoch 150/1000
1946/1946 [==============================] - 0s 66us/step - loss: 0.0015 - val_loss: 2.0819e-04
Epoch 151/1000
1946/1946 [==============================] - 0s 53us/step - loss: 0.0015 - val_loss: 2.0819e-04
Epoch 152/1000
1946/1946 [==============================] - 0s 50us/step - loss: 0.0015 - val_loss: 2.0819e-04
Epoch 153/1000
1946/1946 [==============================] - 0s 58us/step - loss: 0.0015 - val_loss: 2.0819e-04
Epoch 154/1000
1

1946/1946 [==============================] - 0s 62us/step - loss: 0.0015 - val_loss: 2.0801e-04
Epoch 217/1000
1946/1946 [==============================] - 0s 58us/step - loss: 0.0016 - val_loss: 2.0801e-04
Epoch 218/1000
1946/1946 [==============================] - 0s 61us/step - loss: 0.0015 - val_loss: 2.0801e-04
Epoch 219/1000
1946/1946 [==============================] - 0s 39us/step - loss: 0.0015 - val_loss: 2.0801e-04
Epoch 220/1000
1946/1946 [==============================] - 0s 36us/step - loss: 0.0016 - val_loss: 2.0801e-04
Epoch 221/1000
1946/1946 [==============================] - 0s 36us/step - loss: 0.0015 - val_loss: 2.0801e-04

Epoch 00221: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.
Epoch 222/1000
1946/1946 [==============================] - 0s 36us/step - loss: 0.0015 - val_loss: 2.0801e-04
Epoch 223/1000
1946/1946 [==============================] - 0s 45us/step - loss: 0.0015 - val_loss: 2.0801e-04
Epoch 224/1000
1946/1946 [====================

1946/1946 [==============================] - 0s 37us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 288/1000
1946/1946 [==============================] - 0s 46us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 289/1000
1946/1946 [==============================] - 0s 42us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 290/1000
1946/1946 [==============================] - 0s 38us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 291/1000
1946/1946 [==============================] - 0s 50us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 292/1000
1946/1946 [==============================] - 0s 36us/step - loss: 0.0014 - val_loss: 2.0799e-04
Epoch 293/1000
1946/1946 [==============================] - 0s 35us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 294/1000
1946/1946 [==============================] - 0s 41us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 295/1000
1946/1946 [==============================] - 0s 74us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 296/1000
1

Epoch 359/1000
1946/1946 [==============================] - 0s 37us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 360/1000
1946/1946 [==============================] - 0s 37us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 361/1000
1946/1946 [==============================] - 0s 38us/step - loss: 0.0015 - val_loss: 2.0799e-04

Epoch 00361: ReduceLROnPlateau reducing learning rate to 3.814697446813398e-09.
Epoch 362/1000
1946/1946 [==============================] - 0s 37us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 363/1000
1946/1946 [==============================] - 0s 37us/step - loss: 0.0016 - val_loss: 2.0799e-04
Epoch 364/1000
1946/1946 [==============================] - 0s 34us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 365/1000
1946/1946 [==============================] - 0s 35us/step - loss: 0.0014 - val_loss: 2.0799e-04
Epoch 366/1000
1946/1946 [==============================] - 0s 47us/step - loss: 0.0015 - val_loss: 2.0799e-04
Epoch 367/1000
1946/1946 [=====

In [116]:
# 5 refers to days of data. 5 days worth. each 1 row has 36 features
model.predict(np.reshape(ndata[-5:], (5, 1, 36)))

array([[0.0070763 ],
       [0.00162675],
       [0.01396279],
       [0.00855998],
       [0.01180528]], dtype=float32)

In [125]:
import matplotlib.pyplot as plt
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.plot(ndata[:,1])
#Pad the list of predictions to shift it in the graph to it's correct start
for i, data in enumerate(predicted_data):
    padding = [None for p in range(i * prediction_len)]
    plt.plot(padding + data)
    plt.legend()
plt.show()

NameError: name 'predicted_data' is not defined

In [134]:
list(model.predict(np.reshape(ndata[-5:], (5, 1, 36))).reshape(1,-1)[0])

[0.0027149674, -0.00057227165, 0.009761919, 0.0037731677, 0.0063459766]